In [1]:
import openml

In [2]:
openml.config.apikey = "ed68ba028ad40ab0ec6b16791472d3f0"

In [3]:
# SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite

In [6]:
task = benchmark_suite.tasks[0]

In [8]:
task = openml.tasks.get_task(task)  

(1, 10, 1)

In [12]:
for task_id in benchmark_suite.tasks:  # iterate over all tasks
    task = openml.tasks.get_task(task_id)  # download the OpenML task
    dataset = task.get_dataset()
    X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute
    )
    break

In [19]:
task.estimation_procedure_id

1

In [20]:
num_reps, num_folds, num_samples = task.get_split_dimensions()

In [22]:
num_reps, num_folds, num_samples

(1, 10, 1)

In [26]:
task.get_train_test_split_indices(0)

(array([ 1504,  7634,  7907, ..., 34502, 21266, 29826], dtype=int32),
 array([ 3754,  9907, 16444, ..., 31747, 33881, 26981], dtype=int32))

In [23]:
import itertools
jobs = []
for n_fit, (rep_no, fold_no, sample_no) in enumerate(
    itertools.product(
        range(num_reps),
        range(num_folds),
        range(num_samples),
    ),
    start=1,
):
    jobs.append((n_fit, rep_no, fold_no, sample_no))

In [24]:
jobs

[(1, 0, 0, 0),
 (2, 0, 1, 0),
 (3, 0, 2, 0),
 (4, 0, 3, 0),
 (5, 0, 4, 0),
 (6, 0, 5, 0),
 (7, 0, 6, 0),
 (8, 0, 7, 0),
 (9, 0, 8, 0),
 (10, 0, 9, 0)]

In [2]:
import pandas as pd

In [16]:
df = pd.read_csv("benchmark_total.csv")

/tmp/ipykernel_11993/2339968501.py:1: DtypeWarning: Columns (5,8,25,27,31,33,34,52,53,54,55,60,62,67,78,82,86,92,94,98,99,100,101,102,108,110,119,121,128,129,130,131,132,134,136,141,144) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("benchmark_total.csv")


In [17]:
df.head()

,...1,Unnamed: 0,_runtime,_step,_timestamp,_wandb,data__categorical,data__keyword,data__method_name,data__regression,...,...195,...196,...197,...198,...199,...200,...201,...142,...146,...148
0,10.0,10.0,11.961134,0.0,1.672953e+09,{'runtime': 11},0.0,jannis,openml_no_transform,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12.0,12.0,20.292929,0.0,1.672953e+09,{'runtime': 19},0.0,electricity,openml_no_transform,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13.0,13.0,10.445445,0.0,1.672953e+09,{'runtime': 9},0.0,heloc,openml_no_transform,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17.0,17.0,10.317346,0.0,1.672953e+09,{'runtime': 9},0.0,california,openml_no_transform,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18.0,18.0,24.260395,0.0,1.672953e+09,{'runtime': 23},0.0,jannis,openml_no_transform,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# df.loc[
#     (df['data__keyword'] == 'Ailerons'),
#     ].groupby(["model_name","data__method_name"])['max_test_score'].min()

In [19]:
df = df.loc[df.data__method_name == "openml_no_transform"]
df.drop(columns=['data__method_name'], inplace=True)

In [28]:
df = df.loc[df.transformed_target == 0]
df.drop(columns=['transformed_target'], inplace=True)

/tmp/ipykernel_11993/2326729408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['transformed_target'], inplace=True)


In [29]:
df = df.loc[df.model_name != "SAINT"]


In [44]:
reg_tsks = df.loc[
    (df['regression']==1),
    ].groupby(["benchmark","data__keyword", "n_features", "n_train", "model_name"])['mean_test_score'].max()

clf_tsks = df.loc[
    (df['regression']==0),
    ].groupby(["benchmark", "data__keyword", "n_features", "n_train", "model_name"])['mean_test_score'].max()

In [45]:
reg_tsks.to_clipboard()

In [48]:
map = pd.read_clipboard()

In [54]:
reg_tsks = reg_tsks.to_frame().reset_index().merge(map, on='model_name', how='left')
clf_tsks = clf_tsks.to_frame().reset_index().merge(map, on='model_name', how='left')

In [58]:
reg_tsks_an = reg_tsks.groupby(["benchmark", "data__keyword", "n_features", "n_train", "kind"])['mean_test_score'].max()
clf_tsks_an = clf_tsks.groupby(["benchmark", "data__keyword", "n_features", "n_train", "kind"])['mean_test_score'].max()

In [74]:
reg_tsks_an = pd.pivot_table(reg_tsks_an.reset_index(), index=["benchmark", "data__keyword", "n_features", "n_train"], columns=["kind"], values=["mean_test_score"])
reg_tsks_an['diff'] = (reg_tsks_an[('mean_test_score', 'ML')] - reg_tsks_an[('mean_test_score', 'DL')]).abs()

In [76]:
reg_tsks_an.sort_values(by='diff', ascending=True).to_clipboard()

In [77]:
clf_tsks_an = pd.pivot_table(clf_tsks_an.reset_index(), index=["benchmark", "data__keyword", "n_features", "n_train"], columns=["kind"], values=["mean_test_score"])
clf_tsks_an['diff'] = (clf_tsks_an[('mean_test_score', 'ML')] - clf_tsks_an[('mean_test_score', 'DL')]).abs()

In [78]:
clf_tsks_an.sort_values(by='diff', ascending=True).to_clipboard()

In [50]:
df['transform__0__method_name'].unique()

array([nan, 'gaussienize'], dtype=object)

In [48]:
df.loc[
    (df['data__keyword'] == 'compas-two-years')&
    (df['model_name'] == 'XGBoost'),
    ['max_val_score']
    ].max()

max_val_score    0.709172
dtype: float64

In [53]:
df.groupby(["model_name","transform__0__method_name"])["data__keyword"].count()

model_name      transform__0__method_name
FT Transformer  gaussienize                  17463
MLP             gaussienize                  16196
Resnet          gaussienize                  18530
SAINT           gaussienize                  26308
Name: data__keyword, dtype: int64

In [ ]:
df.loc[
    (df['data__method_name'] == 'openml_no_transform'),
    ].groupby(["data__keyword", "model_name", "transformed_target"])['max_test_score'].min()

In [37]:
df[['n_features',
 'n_iter',
 'n_test',
 'n_train']]

,n_features,n_iter,n_test,n_train
0,54.0,auto,33306.0,10000.0
1,7.0,auto,19932.0,10000.0
2,22.0,auto,2100.0,7000.0
3,8.0,auto,7444.0,10000.0
4,54.0,auto,33306.0,10000.0
...,...,...,...,...
380640,359.0,auto,885.0,2946.0
380641,11.0,auto,2246.0,7484.0
380642,4.0,auto,1816.0,6048.0
380643,7.0,auto,852.0,2836.0


In [31]:
df.columns.tolist()

['...1',
 'Unnamed: 0',
 '_runtime',
 '_step',
 '_timestamp',
 '_wandb',
 'data__categorical',
 'data__keyword',
 'data__method_name',
 'data__regression',
 'data_generation_time',
 'hp',
 'max_test_samples',
 'max_train_samples',
 'max_val_samples',
 'mean_r2_test',
 'mean_r2_train',
 'mean_r2_val',
 'mean_test_score',
 'mean_time',
 'mean_train_score',
 'mean_val_score',
 'model__n_estimators',
 'model_id',
 'model_name',
 'model_type',
 'n_features',
 'n_iter',
 'n_test',
 'n_train',
 'one_hot_encoder',
 'processor',
 'regression',
 'sweep_id',
 'sweep_name',
 'train_prop',
 'transformed_target',
 'use_gpu',
 'val_test_prop',
 'max_test_score',
 'max_train_score',
 'max_val_score',
 'min_test_score',
 'min_train_score',
 'min_val_score',
 'std_r2_test',
 'std_r2_train',
 'std_r2_val',
 'std_test_score',
 'std_time',
 'std_train_score',
 'std_val_score',
 'test_scores',
 'times',
 'train_scores',
 'val_scores',
 'early_stopping_rounds',
 'model__n_iter_no_change',
 'model__validation